In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Inaugural speeches of the presidents of the US

In [2]:
speech_df = pd.read_csv('./inaugural_speeches.csv')
print(speech_df.shape)
speech_df.head()

(58, 4)


,Name,Inaugural Address,Date,text
0,George Washington,First Inaugural Address,"Thursday, April 30, 1789",Fellow-Citizens of the Senate and of the House...
1,George Washington,Second Inaugural Address,"Monday, March 4, 1793",Fellow Citizens: I AM again called upon by th...
2,John Adams,Inaugural Address,"Saturday, March 4, 1797","WHEN it was first perceived, in early times, t..."
3,Thomas Jefferson,First Inaugural Address,"Wednesday, March 4, 1801",Friends and Fellow-Citizens: CALLED upon to u...
4,Thomas Jefferson,Second Inaugural Address,"Monday, March 4, 1805","PROCEEDING, fellow-citizens, to that qualifica..."


## Cleaning up your text

In [3]:
# replace all non letter characters with a whitespace
# then change to lower case
speech_df['text_clean'] = speech_df['text'].str.replace(
    '[^a-zA-Z]', ' ').str.lower()

speech_df['text_clean'].head()

0    fellow citizens of the senate and of the house...
1    fellow citizens   i am again called upon by th...
2    when it was first perceived  in early times  t...
3    friends and fellow citizens   called upon to u...
4    proceeding  fellow citizens  to that qualifica...
Name: text_clean, dtype: object

## High level text features

In [7]:
# find the length of each text
speech_df['char_cnt'] = speech_df['text_clean'].str.len()

# count the number of words in each text
speech_df['word_cnt'] = speech_df['text_clean'].str.split().str.len()

# find the average length of words
speech_df['avg_word_length'] = speech_df['char_cnt'] / speech_df['word_cnt']

speech_df[['text_clean', 'char_cnt', 'word_cnt', 'avg_word_length']].head()

,text_clean,char_cnt,word_cnt,avg_word_length
0,fellow citizens of the senate and of the house...,8616,1432,6.016760
1,fellow citizens i am again called upon by th...,787,135,5.829630
2,when it was first perceived in early times t...,13871,2323,5.971158
3,friends and fellow citizens called upon to u...,10144,1736,5.843318
4,proceeding fellow citizens to that qualifica...,12902,2169,5.948363


## Counting words (I)

In [9]:
# Instantiate CountVectorizer
cv = CountVectorizer()

# fit
cv.fit(speech_df['text_clean'])

# print feature names
print(cv.get_feature_names())

['abandon', 'abandoned', 'abandonment', 'abate', 'abdicated', 'abeyance', 'abhorring', 'abide', 'abiding', 'abilities', 'ability', 'abject', 'able', 'ably', 'abnormal', 'abode', 'abolish', 'abolished', 'abolishing', 'aboriginal', 'aborigines', 'abound', 'abounding', 'abounds', 'about', 'above', 'abraham', 'abreast', 'abridging', 'abroad', 'absence', 'absent', 'absolute', 'absolutely', 'absolutism', 'absorb', 'absorbed', 'absorbing', 'absorbs', 'abstain', 'abstaining', 'abstract', 'abstractions', 'absurd', 'abundance', 'abundant', 'abundantly', 'abuse', 'abused', 'abuses', 'academies', 'accept', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessible', 'accession', 'accident', 'accidental', 'accidents', 'acclaim', 'accommodation', 'accommodations', 'accompanied', 'accompany', 'accomplish', 'accomplished', 'accomplishing', 'accomplishment', 'accomplishments', 'accord', 'accordance', 'accorded', 'according', 'accordingly', 'accords', 'account', 'accountability', 'accountab

In [10]:
# apply vectorizer
cv_transformed = cv.transform(speech_df['text_clean'])

# full array
cv_array = cv_transformed.toarray()

print(cv_array.shape)

(58, 9043)


## Limiting your features

In [11]:
# specify arguments to limit the number of features generated
cv = CountVectorizer(min_df=0.2,
                     max_df=0.8)

# fit and transform
cv_transformed = cv.fit_transform(speech_df['text_clean'])
cv_array = cv_transformed.toarray()

print(cv_array.shape)

(58, 818)


## Text to DataFrame

In [14]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [15]:
# create a dataframe with these features
cv_df = pd.DataFrame(cv_array,
                     columns=cv.get_feature_names()).add_prefix('Counts_')

# add new cols to the original df
speech_df_new = pd.concat([speech_df, cv_df], 
                          axis=1,
                          sort=False)
speech_df_new.head()

,Name,Inaugural Address,Date,text,text_clean,char_cnt,word_cnt,avg_word_length,Counts_abiding,Counts_ability,Counts_able,Counts_about,Counts_above,Counts_abroad,Counts_accept,Counts_accomplished,Counts_achieve,Counts_across,Counts_act,Counts_action,Counts_acts,Counts_add,Counts_adequate,Counts_administration,Counts_adopted,Counts_advance,Counts_advantage,Counts_affairs,Counts_afford,Counts_after,Counts_again,Counts_against,Counts_age,Counts_ago,Counts_agriculture,Counts_aid,Counts_alike,Counts_almighty,Counts_almost,Counts_alone,Counts_along,Counts_already,Counts_also,Counts_always,Counts_am,Counts_america,Counts_american,Counts_americans,Counts_among,Counts_ancient,Counts_another,Counts_appear,Counts_armed,Counts_arms,Counts_around,Counts_ask,Counts_assume,Counts_attempt,Counts_attention,Counts_authority,Counts_avoid,Counts_away,Counts_back,Counts_balance,Counts_basis,Counts_bear,Counts_because,Counts_become,Counts_beginning,Counts_being,Counts_belief,Counts_believe,Counts_believed,Counts_belongs,Counts_benefit,Counts_benefits,Counts_best,Counts_better,Counts_between,Counts_beyond,Counts_birth,Counts_bless,Counts_blessings,Counts_blood,Counts_body,Counts_bonds,Counts_born,Counts_both,Counts_bound,Counts_branches,Counts_bring,Counts_brought,Counts_build,Counts_burden,Counts_burdens,Counts_business,Counts_call,Counts_called,Counts_came,Counts_cannot,Counts_capacity,Counts_capital,Counts_care,Counts_carry,Counts_cause,Counts_centuries,Counts_century,Counts_certain,Counts_challenge,Counts_chance,Counts_change,Counts_changes,Counts_character,Counts_cherish,Counts_chief,Counts_children,Counts_choice,Counts_chosen,Counts_circumstances,Counts_citizen,Counts_citizenship,Counts_civil,Counts_civilization,Counts_claim,Counts_class,Counts_clear,Counts_clearly,Counts_close,Counts_come,Counts_command,Counts_commerce,Counts_committed,Counts_common,Counts_communities,Counts_community,Counts_complete,Counts_concern,Counts_concerns,Counts_condition,Counts_conditions,Counts_conduct,Counts_confidence,Counts_conflict,Counts_congress,Counts_conscience,Counts_consequences,Counts_consider,Counts_consideration,Counts_considered,Counts_constant,Counts_constantly,Counts_constitution,Counts_constitutional,Counts_continent,Counts_continue,Counts_continued,Counts_control,Counts_conviction,Counts_cooperation,Counts_cost,Counts_could,Counts_countries,Counts_countrymen,Counts_courage,Counts_course,Counts_create,Counts_created,Counts_credit,Counts_crisis,Counts_danger,Counts_dangers,Counts_day,Counts_days,Counts_debt,Counts_decision,Counts_decisions,Counts_declaration,Counts_declare,Counts_declared,Counts_deep,Counts_deeply,Counts_defend,Counts_defense,Counts_degree,Counts_demand,Counts_demands,Counts_democracy,Counts_departments,Counts_depend,Counts_desire,Counts_destiny,Counts_destroy,Counts_destruction,Counts_determination,Counts_determined,Counts_development,Counts_devotion,Counts_did,Counts_difference,Counts_differences,Counts_different,Counts_difficult,Counts_difficulties,Counts_dignity,Counts_direct,Counts_direction,Counts_discharge,Counts_disposition,Counts_distant,Counts_distinguished,Counts_divine,Counts_does,Counts_doing,Counts_domestic,Counts_done,Counts_doubt,Counts_down,Counts_due,Counts_during,Counts_duties,Counts_duty,Counts_each,Counts_earth,Counts_economic,Counts_economy,Counts_education,Counts_effect,Counts_effective,Counts_effort,Counts_efforts,Counts_either,Counts_election,Counts_employed,Counts_encourage,Counts_encouragement,Counts_end,Counts_endeavor,Counts_enemies,Counts_energy,Counts_engaged,Counts_enjoy,Counts_enjoyed,Counts_enlightened,Counts_enough,Counts_enter,Counts_enterprise,Counts_equal,Counts_equality,Counts_equally,Counts_especially,Counts_essential,Counts_establish,Counts_established,Counts_establishment,Counts_even,Counts_events,Counts_ever,Counts_everywhere,Counts_evil,Counts_example,Counts_except,Counts_execute,Counts_executive,Counts_exercise,Counts_existence,Counts_expect,Counts_expected,Counts_expenditures,Counts_exper

## Tf-idf

In [18]:
# instantiate
tv = TfidfVectorizer(max_features=100, 
                     stop_words='english')

# fit and transform
tv_transformed = tv.fit_transform(speech_df['text_clean'])

# create a df with these features
tv_df = pd.DataFrame(tv_transformed.toarray(),
                     columns=tv.get_feature_names()).add_prefix('TDIDF_')
tv_df.head()

,TDIDF_action,TDIDF_administration,TDIDF_america,TDIDF_american,TDIDF_americans,TDIDF_believe,TDIDF_best,TDIDF_better,TDIDF_change,TDIDF_citizens,TDIDF_come,TDIDF_common,TDIDF_confidence,TDIDF_congress,TDIDF_constitution,TDIDF_country,TDIDF_day,TDIDF_duties,TDIDF_duty,TDIDF_equal,TDIDF_executive,TDIDF_faith,TDIDF_far,TDIDF_federal,TDIDF_fellow,TDIDF_force,TDIDF_foreign,TDIDF_free,TDIDF_freedom,TDIDF_future,TDIDF_general,TDIDF_god,TDIDF_good,TDIDF_government,TDIDF_great,TDIDF_high,TDIDF_history,TDIDF_home,TDIDF_hope,TDIDF_human,TDIDF_institutions,TDIDF_interests,TDIDF_just,TDIDF_justice,TDIDF_know,TDIDF_land,TDIDF_law,TDIDF_laws,TDIDF_let,TDIDF_liberty,TDIDF_life,TDIDF_long,TDIDF_make,TDIDF_man,TDIDF_means,TDIDF_men,TDIDF_nation,TDIDF_national,TDIDF_nations,TDIDF_necessary,TDIDF_need,TDIDF_new,TDIDF_office,TDIDF_old,TDIDF_order,TDIDF_party,TDIDF_peace,TDIDF_people,TDIDF_place,TDIDF_policy,TDIDF_political,TDIDF_power,TDIDF_powers,TDIDF_present,TDIDF_president,TDIDF_principles,TDIDF_progress,TDIDF_prosperity,TDIDF_public,TDIDF_purpose,TDIDF_right,TDIDF_rights,TDIDF_secure,TDIDF_service,TDIDF_shall,TDIDF_spirit,TDIDF_state,TDIDF_states,TDIDF_strength,TDIDF_support,TDIDF_things,TDIDF_time,TDIDF_today,TDIDF_union,TDIDF_united,TDIDF_war,TDIDF_way,TDIDF_work,TDIDF_world,TDIDF_years
0,0.000000,0.133415,0.000000,0.105388,0.0,0.000000,0.000000,0.000000,0.000000,0.229644,0.0,0.000000,0.111079,0.000000,0.060755,0.229644,0.115098,0.064225,0.238637,0.063036,0.147280,0.000000,0.178978,0.000000,0.147528,0.000000,0.000000,0.098352,0.000000,0.101797,0.000000,0.0,0.147528,0.367430,0.133183,0.000000,0.000000,0.000000,0.051787,0.126073,0.000000,0.063036,0.098352,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055540,0.050028,0.000000,0.000000,0.000000,0.064225,0.050898,0.091858,0.052694,0.047521,0.147280,0.00000,0.049176,0.000000,0.000000,0.141458,0.070729,0.000000,0.174590,0.056532,0.138691,0.000000,0.050898,0.065448,0.315182,0.061880,0.063036,0.000000,0.064225,0.333237,0.0,0.055540,0.050898,0.000000,0.063036,0.145021,0.000000,0.000000,0.103573,0.000000,0.000000,0.000000,0.045929,0.0,0.136012,0.203593,0.000000,0.060755,0.000000,0.045929,0.052694
1,0.000000,0.261016,0.266097,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.179712,0.0,0.000000,0.217318,0.000000,0.237725,0.179712,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.192418,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.179712,0.000000,0.233437,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.242128,0.000000,0.000000,0.000000,0.000000,0.170786,0.000000,0.000000,0.000000,0.000000,0.000000,0.246652,0.242128,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.567446,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.199157,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.092436,0.157058,0.073018,0.0,0.000000,0.026112,0.060460,0.000000,0.106072,0.0,0.056125,0.025654,0.196017,0.224501,0.212143,0.026582,0.029665,0.055113,0.058233,0.068028,0.082669,0.027556,0.000000,0.068143,0.000000,0.246496,0.045428,0.000000,0.023510,0.133321,0.0,0.136285,0.339429,0.102528,0.027556,0.029116,0.000000,0.023920,0.058233,0.034014,0.000000,0.022714,0.14352,0.000000,0.000000,0.000000,0.061624,0.000000,0.076961,0.046216,0.073018,0.024339,0.000000,0.029665,0.094039,0.190929,0.097357,0.241448,0.102042,0.00000,0.022714,0.000000,0.000000,0.130678,0.130678,0.121696,0.403213,0.026112,0.000000,0.027556,0.117549,0.030230,0.058233,0.000000,0.058233,0.000000,0.059331,0.153921,0.0,0.025654,0.023510,0.030230,0.058233,0.089313,0.153921,0.090691,0.215280,0.000000,0.116465,0.032030,0.021214,0.0,0.062823,0.070529,0.024339,0.000000,0.000000,0.063643,0.073018
3,0.000000,0.092693,0.000000,0.000000,0.0,0.090942,0.117831,0.045471,0.053335,0.

## Inspecting tf-idf values

In [19]:
# isolate first row to examine
sample_row = tv_df.iloc[0, :]

# top 5 words
sample_row.sort_values(ascending=False).head()

TDIDF_government    0.367430
TDIDF_public        0.333237
TDIDF_present       0.315182
TDIDF_duty          0.238637
TDIDF_citizens      0.229644
Name: 0, dtype: float64

## Transforming unseen data

In [21]:
train_speech_df = speech_df.iloc[:45, :]
test_speech_df = speech_df.iloc[45:, :]

In [24]:
# instantiate
tv = TfidfVectorizer(max_features=100,
                     stop_words='english')

# fit and transform data
tv_transformed = tv.fit_transform(train_speech_df['text_clean'])

# transform test data
test_tv_transformed = tv.transform(test_speech_df['text_clean'])

# create new features for the test set
test_tv_df = pd.DataFrame(test_tv_transformed.toarray(),
                          columns=tv.get_feature_names()).add_prefix('TFIDF_')
test_tv_df.head()

,TFIDF_action,TFIDF_administration,TFIDF_america,TFIDF_american,TFIDF_authority,TFIDF_best,TFIDF_business,TFIDF_citizens,TFIDF_commerce,TFIDF_common,TFIDF_confidence,TFIDF_congress,TFIDF_constitution,TFIDF_constitutional,TFIDF_country,TFIDF_day,TFIDF_duties,TFIDF_duty,TFIDF_equal,TFIDF_executive,TFIDF_faith,TFIDF_far,TFIDF_federal,TFIDF_fellow,TFIDF_force,TFIDF_foreign,TFIDF_free,TFIDF_freedom,TFIDF_future,TFIDF_general,TFIDF_given,TFIDF_god,TFIDF_good,TFIDF_government,TFIDF_great,TFIDF_high,TFIDF_hope,TFIDF_human,TFIDF_important,TFIDF_institutions,TFIDF_interests,TFIDF_just,TFIDF_justice,TFIDF_know,TFIDF_law,TFIDF_laws,TFIDF_let,TFIDF_liberty,TFIDF_life,TFIDF_long,TFIDF_make,TFIDF_man,TFIDF_means,TFIDF_men,TFIDF_nation,TFIDF_national,TFIDF_nations,TFIDF_necessary,TFIDF_new,TFIDF_office,TFIDF_order,TFIDF_ought,TFIDF_party,TFIDF_peace,TFIDF_people,TFIDF_policy,TFIDF_political,TFIDF_power,TFIDF_powers,TFIDF_present,TFIDF_principle,TFIDF_principles,TFIDF_progress,TFIDF_proper,TFIDF_prosperity,TFIDF_protection,TFIDF_public,TFIDF_purpose,TFIDF_question,TFIDF_republic,TFIDF_revenue,TFIDF_right,TFIDF_rights,TFIDF_secure,TFIDF_self,TFIDF_service,TFIDF_shall,TFIDF_spirit,TFIDF_state,TFIDF_states,TFIDF_subject,TFIDF_support,TFIDF_time,TFIDF_union,TFIDF_united,TFIDF_war,TFIDF_way,TFIDF_work,TFIDF_world,TFIDF_years
0,0.000000,0.029540,0.233954,0.082703,0.000000,0.000000,0.000000,0.022577,0.0,0.000000,0.026350,0.000000,0.026950,0.0,0.022577,0.029540,0.000000,0.0,0.065003,0.0,0.031720,0.056409,0.000000,0.049296,0.000000,0.000000,0.049296,0.066626,0.026350,0.000000,0.030968,0.195008,0.024111,0.115378,0.110450,0.055135,0.079050,0.000000,0.000000,0.0,0.000000,0.025200,0.025767,0.341578,0.029540,0.059080,0.348924,0.000000,0.128835,0.055135,0.242549,0.309678,0.03172,0.024111,0.115378,0.025200,0.070760,0.000000,0.201600,0.028861,0.034158,0.0,0.000000,0.316200,0.302600,0.0,0.000000,0.025767,0.000000,0.000000,0.000000,0.000000,0.030968,0.0,0.0,0.0,0.000000,0.029540,0.000000,0.000000,0.0,0.000000,0.000000,0.030242,0.000000,0.0,0.086457,0.165406,0.000000,0.024648,0.0,0.000000,0.115378,0.000000,0.024648,0.079050,0.033313,0.000000,0.299983,0.134749
1,0.000000,0.000000,0.547457,0.036862,0.000000,0.036036,0.000000,0.015094,0.0,0.000000,0.017617,0.000000,0.000000,0.0,0.045283,0.019750,0.000000,0.0,0.021730,0.0,0.084830,0.037714,0.000000,0.016479,0.043459,0.000000,0.000000,0.089089,0.052851,0.000000,0.020704,0.086919,0.016120,0.154278,0.132920,0.018431,0.035234,0.040438,0.043459,0.0,0.000000,0.101089,0.017227,0.000000,0.019750,0.000000,0.466565,0.000000,0.017227,0.073724,0.126126,0.000000,0.00000,0.000000,0.169706,0.016848,0.047308,0.000000,0.252722,0.000000,0.022837,0.0,0.000000,0.334722,0.086705,0.0,0.000000,0.017227,0.018857,0.000000,0.024041,0.000000,0.103522,0.0,0.0,0.0,0.000000,0.019750,0.024685,0.000000,0.0,0.108108,0.016120,0.020219,0.000000,0.0,0.101155,0.036862,0.000000,0.000000,0.0,0.019296,0.092567,0.000000,0.000000,0.052851,0.066817,0.078999,0.277701,0.126126
2,0.000000,0.000000,0.126987,0.134669,0.000000,0.131652,0.000000,0.000000,0.0,0.046997,0.042907,0.000000,0.000000,0.0,0.036763,0.048102,0.045927,0.0,0.052924,0.0,0.103304,0.000000,0.000000,0.000000,0.000000,0.049244,0.040136,0.216981,0.085814,0.000000,0.100853,0.052924,0.078521,0.150301,0.071941,0.089780,0.085814,0.246220,0.000000,0.0,0.050426,0.164138,0.041958,0.166863,0.096203,0.048102,0.206608,0.089780,0.083915,0.044890,0.000000,0.050426,0.00000,0.000000,0.488478,0.041034,0.038407,0.057055,0.369310,0.046997,0.000000,0.0,0.000000,0.042907,0.211174,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.093993,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.043884,0.117781,0.000000,0.054245,0.0,0.000000,0.269339,0.000000,0.040136,0.0,0.000000,0.075151,0.000000,0.080272,0.042907,0.054245,0.096203,0.225452,0.043884
3,0.037094,0.067428,0.267012,0.031463,0.039990,0.061516,0.050085,0.077301,0.0,0.000000,0.000000,0.039990,0.030758,0.0,0.077301,0.134856,0.000000,0.0,0.074188,0.0,0.108

## Using longer n-grams

In [25]:
# instantiate a trigram vectorizer
cv_trigram_vec = CountVectorizer(max_features=100,
                                 stop_words='english',
                                 ngram_range=(3, 3))

# fit and apply trigram vec
cv_trigram = cv_trigram_vec.fit_transform(speech_df['text_clean'])

# print the trigram features
print(cv_trigram_vec.get_feature_names())

['ability preserve protect', 'agriculture commerce manufactures', 'america ideal freedom', 'believe men right', 'best ability preserve', 'best interests country', 'bless god bless', 'bless united states', 'chief justice mr', 'children children children', 'citizens united states', 'civil religious liberty', 'civil service reform', 'commerce united states', 'confidence fellow citizens', 'congress extraordinary session', 'constitution does expressly', 'constitution united states', 'coordinate branches government', 'cultivate friendship nations', 'day task people', 'defend constitution united', 'distinguished guests fellow', 'does expressly say', 'dreams hopes goals', 'economy public expenditure', 'equal exact justice', 'era good feeling', 'executive branch government', 'faithfully execute office', 'fellow citizens assembled', 'fellow citizens called', 'fellow citizens large', 'fellow citizens world', 'form perfect union', 'general welfare secure', 'god bless america', 'god bless god', 'go

## Finding the most common words

In [26]:
# create a df of the featurees
cv_tri_df = pd.DataFrame(cv_trigram.toarray(),
                         columns=cv_trigram_vec.get_feature_names()).add_prefix('Counts_')

# top 5 words
cv_tri_df.sum().sort_values(ascending=False).head()

Counts_constitution united states    20
Counts_people united states          13
Counts_preserve protect defend       10
Counts_mr chief justice              10
Counts_president united states        8
dtype: int64